Code for Figure 3F

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import random
import glob
from operator import itemgetter
import itertools
from itertools import groupby
import seaborn as sns
import matplotlib.ticker as ticker
import sys
import scipy.stats  as stats
pd.set_option('display.max_columns', None)

In [ ]:
tumor_roi_dat = pd.read_csv('new_annotation_Feb2021_KW.txt', delimiter = "\t")
protein_dat=pd.read_csv('ERCC_IgG_norm.csv')
nuc_count = pd.read_csv('nuc_count.csv')

protein_dat=protein_dat.rename({'Unnamed: 0': 'Protein'}, axis=1) 

all_rois=list(protein_dat.columns)[1:]

tumor_core_roi_dict={}
excluded_rois=[]
for loc in range(len(tumor_roi_dat)):
    cur_row = tumor_roi_dat.iloc[loc]
    roi = cur_row['ROI']
    roi_num=roi.split('_')[1]
    if len(roi_num)==1:
        roi = 'ROI_0'+roi_num
    #print(cur_row)
    if roi in all_rois:
        key = (cur_row['Tumor_ID'],cur_row['Core_ID'])
        if key[0]==21 or key[0]==8:
            excluded_rois.append(roi)
        else:
            if key in tumor_core_roi_dict:
                tumor_core_roi_dict[key].append(roi)
            else:
                tumor_core_roi_dict[key] = [roi]            

tumor_ids=list(tumor_roi_dat['Tumor_ID'])
core_ids=list(tumor_roi_dat['Core_ID'])
fish_cores=list(tumor_roi_dat['FISH_core'])

tumor_core_to_shreya_base={(tumor_ids[i],core_ids[i]):fish_cores[i] for i in range(len(tumor_roi_dat))}
tumor_core_to_shreya={}
for k,v in tumor_core_to_shreya_base.items():
    if k in tumor_core_roi_dict.keys():
        tumor_core_to_shreya[k]=v
all_proteins_cores_shreya=list(tumor_core_to_shreya.values())

tumor_roi_dict={}
for k, rois in tumor_core_roi_dict.items():
    tumor=k[0]
    if tumor in tumor_roi_dict.keys():
        tumor_roi_dict[tumor]=tumor_roi_dict[tumor]+rois
    else:
        tumor_roi_dict[tumor]=rois
            
nuc_count=nuc_count.rename({'Unnamed: 0': 'ROI'}, axis=1) 

tumor_protein_dfs=[]
for tumor,rois in tumor_roi_dict.items():
    cur_df=protein_dat[rois]
    cur_df.index=protein_dat['Protein']
    tot_nuc_count=nuc_count[nuc_count['ROI'].isin(protein_dat[rois])]['nuclei_count'].sum()
    for roi in rois:
        cur_weight=list(nuc_count[nuc_count['ROI']==roi]['nuclei_count'])[0]/tot_nuc_count
        cur_df[roi]=cur_df[roi]*cur_weight 
    cur_df=cur_df.sum(axis=1).to_frame().rename(columns={0:tumor})
    tumor_protein_dfs.append(cur_df)
protein_dat_by_tumor=pd.concat(tumor_protein_dfs,1)

protein_dat_by_tumor=protein_dat_by_tumor.T

In [ ]:
len(all_proteins_cores_shreya)

In [ ]:
filenames = glob.glob("FISH_data/table*")
dfs_o = {}
for filename in filenames:
    df_o = pd.read_csv(filename)
    shreya_core_num = int(filename.split('_')[2])
    image_num = int(filename.split('_')[3].split('.')[0])
    dfs_o[(shreya_core_num,image_num)]=df_o

    
HI=6
MAX_HI = 50

def geno_hi(geno,df):
    return (df[geno]>=HI)&(df[geno]<=MAX_HI)
def geno_lo(geno,df):
    return (df[geno]>=1)&(df[geno]<HI)

for df_fish in list(dfs_o.values()):
    df_fish['EGFR_amp']=1*(geno_hi('EGFR',df_fish))
    df_fish['PDGFRA_amp']=1*(geno_hi('PDGFRA',df_fish))
    df_fish['CDK4_amp']=1*(geno_hi('CDK4',df_fish))
    
dfs={}    
for key,df in dfs_o.items():
        
    if key[0] not in tumor_core_to_shreya.values():
        continue

    
    df['HHH']=1*((df['EGFR']>=HI)&(df['PDGFRA']>=HI)&(df['CDK4']>=HI))
    df['HHL']=1*((df['EGFR']>=HI)&(df['PDGFRA']>=HI)&(df['CDK4']<HI))
    df['HLH']=1*((df['EGFR']>=HI)&(df['PDGFRA']<HI)&(df['CDK4']>=HI))
    df['LHH']=1*((df['EGFR']<HI)&(df['PDGFRA']>=HI)&(df['CDK4']>=HI))
    df['HLL']=1*((df['EGFR']>=HI)&(df['PDGFRA']<HI)&(df['CDK4']<HI))
    df['LHL']=1*((df['EGFR']<HI)&(df['PDGFRA']>=HI)&(df['CDK4']<HI))
    df['LLH']=1*((df['EGFR']<HI)&(df['PDGFRA']<HI)&(df['CDK4']>=HI))
    df['LLL']=1*((df['EGFR']<HI)&(df['PDGFRA']<HI)&(df['CDK4']<HI))

    dfs[key]=df[(df['EGFR']<=MAX_HI)&(df['PDGFRA']<=MAX_HI)&(df['CDK4']<=MAX_HI)]

In [ ]:
#TERT
filenames = glob.glob("hTERTmut_count_results/Log_MAX_TMAtert_TMA_*")
tert_files = {}
xy_files = {}

for filename in filenames:
    file_type = filename.split('TMA')[-1].split('_')[-1].split('.')[0]
    core_image = filename.split('TMA')[-1].split('__')[0].split('_')
    core = int(core_image[1])
    image = int(core_image[2])
    if file_type =='tert':
        tert_files[(core,image)]=filename
    elif file_type == 'xyarea':
        xy_files[(core,image)]=filename
        
dfs_TERT={}
for key in list(tert_files.keys()):
    if key in list(xy_files.keys()):
        
        if key[0] not in tumor_core_to_shreya.values():
            continue
        
        df_tert = pd.read_csv(tert_files[key],header=None)
        df_xy = pd.read_csv(xy_files[key],header=None)

        df_xy=df_xy.set_index(0)
        df_xy=df_xy.rename(columns={1:'X',2:'Y'})
        df_xy = df_xy.drop(3,1)

        df_tert=df_tert.set_index(0)
        df_tert=df_tert.rename(columns={1:'TERT_count'})

        df=pd.concat([df_xy,df_tert],axis=1, sort=False)
        df['TERT_bool']=1*(df['TERT_count']>0)
        if len(df)>10:
            dfs_TERT[key]=df



In [ ]:
len(dfs_TERT)

In [ ]:
TERT_core_to_images={}
for k in dfs_TERT.keys():
    this_core=k[0]
    if this_core in TERT_core_to_images.keys():
        TERT_core_to_images[this_core]=TERT_core_to_images[this_core]+[k]
    else:
        TERT_core_to_images[this_core]=[k]

In [ ]:
dfs_TERT_in_core={}
for core, images in TERT_core_to_images.items():
    dfs_TERT_in_core[core]=pd.concat([dfs_TERT[image] for image in images])

In [ ]:
len(dfs_TERT_in_core)

In [ ]:
len(set([item[0] for item in dfs_TERT.keys()]))

In [ ]:
prop_TERT=[]
for core,df_TERT_in_core in dfs_TERT_in_core.items():
    this_prop_TERT=100*len(df_TERT_in_core[df_TERT_in_core['TERT_bool']==1])/len(df_TERT_in_core)
    prop_TERT.append(this_prop_TERT)

In [ ]:
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = "28"
fig = plt.figure(figsize=(6.1,7))
sns.distplot(prop_TERT,kde=False,color='darkorchid')
plt.ylabel('Number of cores')
plt.xlabel('Frequency (%)\nof TERT-mutant cells')
plt.xticks([0,20,40,60,80])
plt.yticks([0,2,4,6,8,10,12])

plt.tight_layout()
#plt.savefig('TERT_core_props.pdf')
plt.show()